<a href="https://colab.research.google.com/github/shailza969/Dog-and-cat-classifiaction/blob/main/Dog_%26_Cat_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shailzabansal","key":"1b2411f7858f8e80b31e0e2385161061"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d chetankv/dogs-cats-images

 98% 425M/435M [00:10<00:00, 31.2MB/s]
100% 435M/435M [00:10<00:00, 44.6MB/s]


In [ ]:
! unzip /content/dogs-cats-images.zip
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras.preprocessing import image
from tqdm import tqdm

In [ ]:
path = '/content/dog vs cat/dataset'
X_train = []
y_train = []
X_test = []
y_test = []

In [ ]:
def prep_train (path, label):
  img = image.load_img(path, target_size = (100,100))
  img = image.img_to_array(img)
  X_train.append(img)
  y_train.append(label)

In [ ]:
def prep_test (path, label):
  img = image.load_img(path, target_size = (100,100))
  img = image.img_to_array(img)
  X_test.append(img)
  y_test.append(label)

In [ ]:
#/content/dog vs cat/dataset/training_set
for folder in os.listdir(path):
  for fol in os.listdir(path+'/'+folder):
    if (folder == 'training_set'):
      for img in os.listdir(path+'/'+folder+'/'+ fol):
        prep_train(path+'/'+folder+'/'+ fol+'/'+img, fol)
    else:
      for img in os.listdir(path+'/'+folder+'/'+ fol):
        prep_test(path+'/'+folder+'/'+ fol+'/'+img, fol)

In [ ]:
len(X_train), len(y_train), len(X_test), len(y_test)

(8000, 8000, 2000, 2000)

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)


In [ ]:
X_train.shape, y_train.shape

((8000, 100, 100, 3), (8000,))

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_y = LabelEncoder()

In [ ]:
y_train = le_y.fit_transform(y_train)
y_test = le_y.transform(y_test)

In [ ]:
y_train[:10]
le_y.classes_

array(['cats', 'dogs'], dtype='<U4')

# NN

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten

In [ ]:
clf = Sequential()

In [ ]:
clf.add(Conv2D(32, (3,3), strides=(2,2), activation='relu', input_shape = (100,100,3)))
clf.add(Conv2D(32, (4,4), strides=(2,2), activation='relu'))
clf.add(MaxPool2D(pool_size=(2,2)))
clf.add(Dropout(rate = 0.2))

In [ ]:
clf.add(Conv2D(32, (3,3),strides=(2,2), activation='relu'))
clf.add(Conv2D(32, (2,2), strides=(2,2), activation='relu'))
clf.add(MaxPool2D(pool_size=(2,2)))
clf.add(Dropout(rate = 0.2))

In [ ]:
clf.add(Flatten())

In [ ]:
clf.add(Dense(64, activation='relu'))
clf.add(Dense(1, activation ='sigmoid'))

In [ ]:
clf.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
clf.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 49, 49, 32)        896       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 23, 23, 32)        16416     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 11, 11, 32)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 5, 5, 32)          9248      
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 2, 2, 32)          4128      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 1, 1, 32)         

In [ ]:
clf.fit(X_train, y_train, batch_size= 128, epochs=50, validation_split = 0.2)

Epoch 1/50
50/50 [==============================] - 1s 24ms/step - loss: 0.1825 - accuracy: 0.9309 - val_loss: 1.8093 - val_accuracy: 0.4900
Epoch 2/50
50/50 [==============================] - 1s 21ms/step - loss: 0.1359 - accuracy: 0.9513 - val_loss: 2.0164 - val_accuracy: 0.4825
Epoch 3/50
50/50 [==============================] - 1s 21ms/step - loss: 0.1500 - accuracy: 0.9448 - val_loss: 1.8681 - val_accuracy: 0.5312
Epoch 4/50
50/50 [==============================] - 1s 20ms/step - loss: 0.1576 - accuracy: 0.9403 - val_loss: 2.4801 - val_accuracy: 0.4319
Epoch 5/50
50/50 [==============================] - 1s 21ms/step - loss: 0.1403 - accuracy: 0.9477 - val_loss: 2.3407 - val_accuracy: 0.4375
Epoch 6/50
50/50 [==============================] - 1s 21ms/step - loss: 0.1287 - accuracy: 0.9495 - val_loss: 2.7708 - val_accuracy: 0.4462
Epoch 7/50
50/50 [==============================] - 1s 21ms/step - loss: 0.1336 - accuracy: 0.9495 - val_loss: 2.6257 - val_accuracy: 0.4313
Epoch 8/50
50

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
clf.evaluate(X_test, y_test)

63/63 [==============================] - 0s 4ms/step - loss: 1.5121 - accuracy: 0.6430


[1.5121039152145386, 0.6430000066757202]